# ORM

- We now cross into a new dimension of everything thus far, using the ORM.



In [353]:
from datetime import datetime
from sqlalchemy import create_engine, func
from sqlalchemy.orm import Mapped, mapped_column, DeclarativeBase, MappedAsDataclass


class Base(MappedAsDataclass, DeclarativeBase):
    pass


class User(Base):  # class can be called whatever we want, no impact on the table name
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True, init=False)
    name: Mapped[str]
    full_name: Mapped[str | None]
    created_at: Mapped[datetime] = mapped_column(init=False, server_default=func.now())


engine = create_engine("sqlite://", echo=True)
with engine.begin() as conn:
    Base.metadata.create_all(conn)

2024-12-09 15:33:42,029 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-09 15:33:42,030 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-12-09 15:33:42,030 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-09 15:33:42,031 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-12-09 15:33:42,031 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-09 15:33:42,032 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	full_name VARCHAR, 
	created_at DATETIME DEFAULT (CURRENT_TIMESTAMP) NOT NULL, 
	PRIMARY KEY (id)
)


2024-12-09 15:33:42,033 INFO sqlalchemy.engine.Engine [no key 0.00139s] ()
2024-12-09 15:33:42,034 INFO sqlalchemy.engine.Engine COMMIT


- We've already been working with ORM-Centric Metadata models, but weren't really using "the ORM".
- To use real ORM-centric patters, we'll add the use of **instances** of our mappes classes.
- Both are super important in the ORM - SQLAlchemy add rich behaviours to both classes AND instances of those classes

### The Session

- To save and retrieve data from a database, we use a special tool that acts as a bridge between your objects (instances), SQL queries (DML statements), and the rows returned by the database.
- It ensures all these actions happen within an active database transaction.
- This object is the `Session` : the `Session` is to the ORM what the `Connection` is to the code - the thing that interacts with the transaction.
- In a similar way as `Connection` objects come from a factory called `Engine`, the `Session` comes from a factory called `sessionMaker()`
- `sessionmaker()` is instanciated with an `Engine`.
- This `Engine` is passed along to `Session` objects, which then use it to get database connections behind the scenes.

In [354]:
from sqlalchemy.orm import sessionmaker

session_factory = sessionmaker(bind=engine)

- Like `Engine`, `session_factory` aims to be a "long-lived", "**application scoped**" variable accross your program. You create it once, an then using it everywhere.
- The `sessionmaker()` creates a `Session` object when we just call it, like a callable:

In [355]:
session = session_factory()  # it isn't connected to a database yet
print(session)

### Session is connected, now what?

- The `Session` connects to database engines **on demand**, after it was instanciated.
- Once the `Session` has established a connection, it's considered to be **in a transaction**.
- The same `Connection` object will continue to be used until the transaction ends.
- We can run any Core or ORM SQL satement using methods like `Session.execute()` or `Session.scalars()`. We can still use `Select` but `Session` will do more special things (like returning objects)

In [356]:
from sqlalchemy import text

result = session.execute(text("select 'hello'"))
print(result)

2024-12-09 15:33:42,064 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-09 15:33:42,066 INFO sqlalchemy.engine.Engine select 'hello'
2024-12-09 15:33:42,066 INFO sqlalchemy.engine.Engine [generated in 0.00058s] ()


- `Session.execute()` returns a `Result` object, just like `Connexion.execute()` does as well.
- To end the transaction and **release** the `Connection`, call `Session.commit()`, `Session.rollback()`, or `Session.close()`

### Properly managing Session scope

- Since `Session` has connect/close and begin/commit cycles, we want to use Python conext manager pattern again:


In [357]:
from sqlalchemy import select

with session_factory() as sess:
    print(sess.execute(select(User.id)).all())

2024-12-09 15:33:42,073 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-09 15:33:42,075 INFO sqlalchemy.engine.Engine SELECT user_account.id 
FROM user_account
2024-12-09 15:33:42,075 INFO sqlalchemy.engine.Engine [generated in 0.00047s] ()
[]
2024-12-09 15:33:42,076 INFO sqlalchemy.engine.Engine ROLLBACK


- `sessionmaker() / Session` supports the same transaction patterns as `Engine / Connection`
1. "commit as you go" where we call `Session.commit()` any number of times:

In [358]:
from sqlalchemy import insert


with session_factory() as sess:
    sess.execute(
        insert(User),
        [
            {"name": "Sandy", "full_name": "Sandy Kilo"},
            {"name": "Gary", "full_name": "Gary Gareau"},
        ],
    )
    sess.commit()

2024-12-09 15:33:42,082 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-09 15:33:42,084 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, full_name) VALUES (?, ?)
2024-12-09 15:33:42,084 INFO sqlalchemy.engine.Engine [generated in 0.00031s] [('Sandy', 'Sandy Kilo'), ('Gary', 'Gary Gareau')]
2024-12-09 15:33:42,085 INFO sqlalchemy.engine.Engine COMMIT


2. "begin once" where we call `sessionmaker.begin()` which establishes a new `Session` and provides a transaction-committing block:


In [359]:
from sqlalchemy import delete

with session_factory.begin() as sess:
    sess.execute(delete(User))

2024-12-09 15:33:42,090 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-09 15:33:42,091 INFO sqlalchemy.engine.Engine DELETE FROM user_account
2024-12-09 15:33:42,092 INFO sqlalchemy.engine.Engine [generated in 0.00038s] ()
2024-12-09 15:33:42,092 INFO sqlalchemy.engine.Engine COMMIT


### Unit of work patterns with the Session

- For the purpose of the following examples, we will use a session without context manager, but this is only for example.

In [360]:
session = session_factory()  # we should not do this

# Let's build an object
dave = User(name="Dave", full_name="Daveey Dee")
print(dave)

# We want to persist the data into a new row in the "user_account" table
# For this, we use Session.add()
session.add(dave)

User(id=None, name='Dave', full_name='Daveey Dee', created_at=None)


- This did not yet modify the database, but the object is now referred towards as **pending**. It means "this is a Python oject that will be used to populate an `INSERT` statement.
- We can see the pending objects by looking at the `session.new` attribute:

In [361]:
session.new

IdentitySet([User(id=None, name='Dave', full_name='Daveey Dee', created_at=None)])

- The process by which the `Session`emits `INSERT`, `UPDATE`, and `DELETE` statements for objects such as these is known as the **flush**
- The flush process occurs:
    - when we run an SQL statement with `Session.execute()` or similar, before that SQL statement is executed (known as **autoflush**)
    - when any ORM instance runs a process known as "lazy loading" (also part of autoflash; more on that later)
    - when we call an explicit method `Session.flush()`
    - when we commit the transaction with Session.commit(), before the actual `COMMIT` occurs.

In [362]:
select_stmt = select(User).where(User.name == "Dave")
result = session.execute(select_stmt)  # insert of previously added user will occur here

2024-12-09 15:33:42,111 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-09 15:33:42,113 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, full_name) VALUES (?, ?) RETURNING id, created_at
2024-12-09 15:33:42,114 INFO sqlalchemy.engine.Engine [generated in 0.00066s] ('Dave', 'Daveey Dee')
2024-12-09 15:33:42,115 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.full_name, user_account.created_at 
FROM user_account 
WHERE user_account.name = ?
2024-12-09 15:33:42,115 INFO sqlalchemy.engine.Engine [generated in 0.00040s] ('Dave',)


The `Result` that we get back from `Session.execute()` has a single row

In [363]:
print(result)
row = result.first()  # the result has a single row
print(row)

if row:
    print(isinstance(row[0], User))

(User(id=1, name='Dave', full_name='Daveey Dee', created_at=datetime.datetime(2024, 12, 9, 15, 33, 42)),)
True


- On this `User` object, we see that server generated attributes `.id` and `.created_at`
- The `User` object we got back is also the same Python object in memory as the original one we used with `Session.add()`:

In [364]:
if row:
    print(id(dave) == id(row[0]))

print(dave)  # as a consequence, we can see that `dave` is also updated!

True
User(id=1, name='Dave', full_name='Daveey Dee', created_at=datetime.datetime(2024, 12, 9, 15, 33, 42))


### The Identity Map

- The way we got back the same object as the one we added is because the `Session` used an **Identity Map**
- An Identity Map is a design pattern used to ensure that each object in your program corresponds to exactly one database record during a single transaction or session. It prevents multiple instances of the same entity from being loaded into memory, maintaining a "one-to-one correspondence" between the in-memory representation and the database row.

In [365]:
dict(session.identity_map)

{(__main__.User,
  (1,),
  None): User(id=1, name='Dave', full_name='Daveey Dee', created_at=datetime.datetime(2024, 12, 9, 15, 33, 42))}

- All objects that are persisted bu the `Session` using flush, as well as all objects that we load using `Session.scalars()` or `Session.execute()`,etc are place in the identity map.
- An object that is present in the identity map is said to be in a **persistent** state.
- This means: "there is a row in the current database transaction that matches this objects primary key identity".

### Selecting objects with scalars

- Remember how we had to extract our `User` object from a row?

In [366]:
row[0]  # type: ignore

User(id=1, name='Dave', full_name='Daveey Dee', created_at=datetime.datetime(2024, 12, 9, 15, 33, 42))

- We will often want to load objects alone from our `SELECT` statements, not rows.
- So, for everyday ORM "load objects" use, we will use `Session.scalars()` (always return Python objects) more often than `Session.execute()` (always return rows)

In [367]:
my_dave_user = session.scalars(select(User).where(User.name == "Dave")).first()
print(my_dave_user)

2024-12-09 15:33:42,263 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.full_name, user_account.created_at 
FROM user_account 
WHERE user_account.name = ?
2024-12-09 15:33:42,264 INFO sqlalchemy.engine.Engine [cached since 0.1487s ago] ('Dave',)
User(id=1, name='Dave', full_name='Daveey Dee', created_at=datetime.datetime(2024, 12, 9, 15, 33, 42))


### Making changes

`Session.add_all()` lets is ad more objects into the pending state.

In [368]:
session.add_all([User("patrick", "Patrick Fiori"), User("Tom", "Cook")])

print(session.new)  # as before, we can these pending objects in Session.new

IdentitySet([User(id=None, name='patrick', full_name='Patrick Fiori', created_at=None), User(id=None, name='Tom', full_name='Cook', created_at=None)])


- For objects that are **already persistent**, we can modify their attributes:

In [369]:
dave.name = "Dayyyyve"

- Persistent objects that have Python-side changes on them are referred as **dirty**
- They can be seen with `Session.dirty`

In [370]:
print(session.dirty)

IdentitySet([User(id=1, name='Dayyyyve', full_name='Daveey Dee', created_at=datetime.datetime(2024, 12, 9, 15, 33, 42))])


In [371]:
session.commit()
print(session.new)
print(session.dirty)

2024-12-09 15:33:42,288 INFO sqlalchemy.engine.Engine UPDATE user_account SET name=? WHERE user_account.id = ?
2024-12-09 15:33:42,289 INFO sqlalchemy.engine.Engine [generated in 0.00083s] ('Dayyyyve', 1)
2024-12-09 15:33:42,289 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, full_name) VALUES (?, ?) RETURNING id, created_at
2024-12-09 15:33:42,290 INFO sqlalchemy.engine.Engine [generated in 0.00005s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('patrick', 'Patrick Fiori')
2024-12-09 15:33:42,291 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, full_name) VALUES (?, ?) RETURNING id, created_at
2024-12-09 15:33:42,291 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('Tom', 'Cook')
2024-12-09 15:33:42,291 INFO sqlalchemy.engine.Engine COMMIT
IdentitySet([])
IdentitySet([])
